In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [59]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, user_knn as kn, item_knn as iknn 

In [4]:
import numpy as np
import pandas as pd
%matplotlib inline

In [23]:
dataset = os.path.join(parentdir,'Book_Data/ratings.csv')     
ratings = pd.read_csv(dataset, sep=',',
                      names=['user', 'item'], header=0)





In [47]:
dataset_items = os.path.join(parentdir,'Book_Data/book_author_gender.csv') 
items = pd.read_csv(dataset_items, sep=',',
                      names=['item', 'author'], header=0)

In [48]:
#items.head()
items = items.loc[(items["author"] == "male") | (items["author"] == "female")]
items.head()

,item,author
0,113484902,male
2,100940351,male
5,104633344,female
6,202193849,male
8,101354697,male


In [29]:
ratings = ratings.iloc[:1000,]
ratings.head()

,user,item
0,2,101569386
1,7,919277058
2,8,100086727
3,8,100227162
4,8,101149545


In [24]:
algo_implicitALS = als.ImplicitMF(50)
algo_als = als.BiasedMF(50)

In [25]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [27]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item']], 2, xf.SampleFrac(0.4)):
    test_data.append(test)
    all_recs.append(eval('Implicit_ALS', algo_implicitALS, train, test))
    #all_recs.append(eval('ALS', algo_als, train, test))

AttributeError: 'DataFrame' object has no attribute 'rating'

In [30]:
all_recs

[              item     score    user  rank     Algorithm
 0        102130480  0.445258       9     1  Implicit_ALS
 1        100151275  0.412746       9     2  Implicit_ALS
 2        101738508  0.391639       9     3  Implicit_ALS
 3        101111047  0.385581       9     4  Implicit_ALS
 4        114040647  0.378191       9     5  Implicit_ALS
 5        100857372  0.362799       9     6  Implicit_ALS
 6        103286596  0.353623       9     7  Implicit_ALS
 7        112161757  0.348910       9     8  Implicit_ALS
 8        101049705  0.347708       9     9  Implicit_ALS
 9        106675606  0.340897       9    10  Implicit_ALS
 10       100338195  0.335564       9    11  Implicit_ALS
 11       104087967  0.321628       9    12  Implicit_ALS
 12       100858196  0.267756       9    13  Implicit_ALS
 13       109368878  0.231584       9    14  Implicit_ALS
 14       100891252  0.210687       9    15  Implicit_ALS
 15       113233458  0.208919       9    16  Implicit_ALS
 16       1125

In [31]:
all_recs = pd.concat(all_recs, ignore_index=True)
test_data = pd.concat(test_data, ignore_index=True)

In [52]:
all_recs_joined = all_recs.join(items[["item", "author"]].set_index("item"), on='item')
#all_recs_joined['Gender'] = np.where(all_recs_joined["name"].str.contains("(M)",case=False, regex=True),"Male","Female")
dummies = pd.get_dummies(all_recs_joined.author)
all_recs_joined = pd.concat([all_recs_joined, dummies], axis=1)
#all_recs_joined.head(20)

In [53]:
providers = ["male", "female"]
items_n = all_recs_joined.item.nunique()
proItems_n =  all_recs_joined.loc[ all_recs_joined['female'] == 1].item.nunique()

In [57]:
items_n

9220

In [55]:
rlaFair = topnFair.FairRecListAnalysis(['user', "Algorithm"])
rlaFair.add_metric("rND")
rlaFair.add_metric("rKL")
rlaFair.add_metric("rRD")
rlaFair.add_metric("div")
rlaFair.add_metric("dem_parity")
rlaFair.add_metric("ndcg")
results = rlaFair.compute(all_recs_joined, test_data, "female", providers, items_n ,proItems_n)
results.head()

,,rND,rKL,rRD,div,dem_parity,ndcg
user,Algorithm,,,,,,
9,Implicit_ALS,0.650567,0.863305,1.224998,1.0,0.131125,0.000000
10,Implicit_ALS,0.544517,0.632206,1.079523,1.0,0.197664,0.000000
16,Implicit_ALS,0.558547,0.589183,1.110757,1.0,0.184167,0.212746
22,Implicit_ALS,0.380401,0.258473,0.845078,1.0,0.305240,0.000000
26,Implicit_ALS,0.220343,0.085245,0.490962,1.0,0.421687,0.000000
